In [1]:
# Import libraries
import torch
import time
from environment import PruningEnv
from REINFORCE_agent import REINFORCE_agent
import os
import logging
import numpy as np
logging.basicConfig(level=logging.INFO, 
                    format=('%(levelname)s:' +
                            '[%(filename)s:%(lineno)d]' +
                            ' %(message)s'))

# Define Agent, Training Env, & Hyper-params
env = PruningEnv()
print(env.state_size)
agent = REINFORCE_agent(env.state_size, 512)

M = 20# no reason, number of training episodes
layers_to_prune = [] # will be list of string names
for layer_name, _ in env.model.named_modules():
    if "conv" in layer_name:
        layers_to_prune.append(layer_name)
listacc = np.zeros([4,M])

listnumpruned = np.zeros([4,M])
listreward = np.zeros([4,M])
listflop = np.zeros([4,M])
listvalacc = np.zeros([100])

env._evaluate_model()

Files already downloaded and verified
Files already downloaded and verified
Starting Pre-Training
Training CNN model
Training Done
512
Evaluating CNN model
Validation Accuracy: 12.00%


0.12

In [2]:
print(env._evaluate_model())

Evaluating CNN model
Validation Accuracy: 12.00%
0.12


In [3]:

for episode in range(M):
    print("=====",episode,"New Episode===========================================================")
    env.reset_to_k() # reset CNN to full-params
    env._evaluate_model()
    action_reward_buffer = [] # list of (action,reward) tuples per episode

    # single rollout, layer-by-layer CNN scan
    for layer_name in layers_to_prune:
        env.layer_to_process = layer_name
        print("===== Working on", layer_name, "layer =====")
        # get state from orig model (or should we get from pruned model?)
        state = env.get_state()
        
        action = agent.get_action(state)


        action = torch.unsqueeze(action, 0)
        #logging.info("Actions: {}".format(action))
        action_to_index = (action > 0.5).type(torch.int)
        #logging.info("action_to_index sum: {}".format(action_to_index.sum()))
        env.prune_layer(action_to_index)
        print("Calculating reward")
        total_filters = 64*(2**(int(layer_name[-1])-1))
        amount_pruned = action_to_index[0,0:64*(2**(int(layer_name[-1])-1))].sum()
        reward,acc,flop = env._calculate_reward(amount_pruned,total_filters)
        
        #reward = reward * -1
        #it seems this doesnt matter? as long as it heads to 0 then that is the goal maybe this is a lr problem
        action_reward_buffer.append((action, reward))

        
        print("For",layer_name,"pruned",action_to_index[0,0:64*(2**(int(layer_name[-1])-1))].sum())
        listacc[int(layer_name[-1])-1, episode]=acc
        print("FILT",64*(2**(int(layer_name[-1])-1)))
        listnumpruned[int(layer_name[-1])-1, episode] = action_to_index[0,0:64*(2**(int(layer_name[-1])-1))].sum()
        listreward[int(layer_name[-1])-1, episode] = reward
        listflop[int(layer_name[-1])-1, episode] = flop
    # calc cumulative reward, agent learns 
    actions, rewards = zip(*action_reward_buffer) # both var are tuple wrapped
    # actions: tuple->tensor
    actions = torch.squeeze(torch.stack(actions)).type(torch.float)
    agent.update_policy(rewards, actions) 


#PATH = os.getcwd() + '\pruned_50_1_train_after.pth'
#torch.save(env.model.state_dict(), PATH)


===== 0 New Episode===========================================================
Evaluating CNN model
Validation Accuracy: 53.81%
===== Working on conv1 layer =====


INFO:[environment.py:126] Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))


Calculating reward
Training CNN model
Training Done
Evaluating CNN model
Validation Accuracy: 40.14%
Reward: tensor(-7.0919, dtype=torch.float64)
For conv1 pruned tensor(41)
FILT 64
===== Working on conv2 layer =====


INFO:[environment.py:126] Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))


Calculating reward
Training CNN model
Training Done
Evaluating CNN model
Validation Accuracy: 13.10%
Reward: tensor(-15.0488, dtype=torch.float64)
For conv2 pruned tensor(64)
FILT 128
===== Working on conv3 layer =====


...

INFO:[environment.py:126] Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))


Calculating reward
Training CNN model
Training Done
Evaluating CNN model
Validation Accuracy: 16.45%
Reward: tensor(-14.4752, dtype=torch.float64)
For conv3 pruned tensor(127)
FILT 256
===== Working on conv4 layer =====


INFO:[environment.py:126] Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))


Calculating reward
Training CNN model
Training Done
Evaluating CNN model
Validation Accuracy: 16.91%
Reward: tensor(-16.7215, dtype=torch.float64)
For conv4 pruned tensor(247)
FILT 512
Gt = [-51.50737692 -45.17119523 -31.02943204 -16.72146916]


INFO:[REINFORCE_agent.py:82] Actions: tensor([[0.5457, 0.6259, 0.8764,  ..., 0.3854, 0.2445, 0.8073],
        [0.5548, 0.5876, 0.5741,  ..., 0.4157, 0.4199, 0.6039],
        [0.4948, 0.5959, 0.5081,  ..., 0.4047, 0.4351, 0.5554],
        [0.5076, 0.5303, 0.5511,  ..., 0.4338, 0.4166, 0.5689]],
       grad_fn=<SqueezeBackward0>)


In [4]:

print("List of accuraccies",listacc)
print("list of pruned", listnumpruned)
print("list of rewards", listreward)
print("list of flops", listflop)
print("list of valacc", listvalacc)

List of accuraccies [[0.4014 0.3952 0.3791 0.3791 0.3791 0.3791 0.3791 0.3791 0.3918 0.3918
  0.3918 0.4109 0.4109 0.412  0.4018 0.4117 0.4249 0.4249 0.4249 0.4249]
 [0.131  0.1238 0.1152 0.1168 0.1168 0.1168 0.1168 0.1168 0.1254 0.1227
  0.1258 0.1351 0.134  0.1337 0.1315 0.1534 0.1605 0.1647 0.1647 0.1647]
 [0.1249 0.118  0.1061 0.1093 0.1082 0.1082 0.1068 0.1069 0.1147 0.1146
  0.1149 0.1178 0.1207 0.1278 0.1263 0.1527 0.1532 0.1634 0.1668 0.1645]
 [0.1161 0.1126 0.1021 0.1046 0.104  0.1052 0.1031 0.103  0.1065 0.1083
  0.1078 0.1103 0.113  0.1205 0.1216 0.1518 0.1449 0.1562 0.1631 0.1691]]
list of pruned [[ 41.  40.  39.  39.  39.  39.  39.  39.  40.  40.  40.  41.  41.  41.
   40.  41.  42.  42.  42.  42.]
 [ 64.  62.  59.  62.  62.  62.  61.  61.  61.  61.  62.  63.  63.  61.
   61.  64.  64.  65.  65.  65.]
 [119. 115. 119. 116. 117. 116. 119. 118. 122. 123. 125. 121. 118. 120.
  121. 122. 118. 123. 127. 127.]
 [251. 248. 245. 246. 244. 248. 252. 251. 251. 249. 249. 247. 244. 24